In [2]:
import os
from dotenv import load_dotenv
import requests

# Load environment variables
load_dotenv()

api_key = os.getenv("STEAM_API_KEY")

# Example API request (get player summaries)
steam_id = os.getenv("STEAM_ID")
url = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={api_key}&steamids={steam_id}"

response = requests.get(url)
data = response.json()

print(data)


{'response': {'players': [{'steamid': '76561199690294419', 'communityvisibilitystate': 3, 'profilestate': 1, 'personaname': 'brendan.rb', 'profileurl': 'https://steamcommunity.com/profiles/76561199690294419/', 'avatar': 'https://avatars.steamstatic.com/7ae537b6b7b7b09e68f52e76cdb8f0727f5cb270.jpg', 'avatarmedium': 'https://avatars.steamstatic.com/7ae537b6b7b7b09e68f52e76cdb8f0727f5cb270_medium.jpg', 'avatarfull': 'https://avatars.steamstatic.com/7ae537b6b7b7b09e68f52e76cdb8f0727f5cb270_full.jpg', 'avatarhash': '7ae537b6b7b7b09e68f52e76cdb8f0727f5cb270', 'lastlogoff': 1762918447, 'personastate': 1, 'primaryclanid': '103582791429521408', 'timecreated': 1716259251, 'personastateflags': 0}]}}
